# SSRS

In [ ]:
# %load imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import seaborn as sns
width=20
height=3
#plt.rcParams["figure.figsize"] = (width,height)
#sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import dask.dataframe
import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from d2e2f.visualization.visualize import plot_map, plot_trips

In [ ]:
%reload_kedro
#ship = 'SSRS15-01_322012079' # west coast
#ship = 'SSRS15-04_322012068' #(bad)
ship = 'SSRS15-03_322012059' # east coast

data_selected = catalog.load("ssrs.data_selected")[ship]()

datas_moving = catalog.load("ssrs.data_moving")
data_moving = datas_moving[ship]()

datas_running = catalog.load("ssrs.data_running")
data_running = datas_running[ship]()

In [ ]:
statistics = catalog.load("ssrs.statistics")[ship]()
statistics_moving = catalog.load("ssrs.statistics_moving")[ship]()
statistics_running = catalog.load("ssrs.statistics_running")[ship]()

In [ ]:
plot_map(df=data_moving, zoom_start=9, color_key='sog', colormap=['black','red'])

In [ ]:
data_running['sog'].mean()

In [ ]:
sns.pairplot(data_moving.drop(columns=['latitude','longitude','cog']).resample('100S').mean(), 
             kind='scatter', plot_kws={'alpha':0.1})


In [ ]:
sns.pairplot(data_running.drop(columns=['latitude','longitude','cog']).resample('100S').mean(), 
             kind='scatter', plot_kws={'alpha':0.1})

In [ ]:
statistics

In [ ]:
statistics_moving

In [ ]:
statistics_running

In [ ]:
from sklearn.svm import SVR
model = LinearRegression()

def features(data):
    X = pd.DataFrame(index=data.index)
    X['sog'] = data['sog']
    X['sog**2'] = data['sog']**2
    X['sog**3'] = data['sog']**3
    X['sog**4'] = data['sog']**4
    X['sog**5'] = data['sog']**5
    X['sog**6'] = data['sog']**6
    
    return X

data = data_running


X = features(data)
#y = data['fuel_per_m']
y = data['fuel_rate']
model.fit(X,y)

In [ ]:
X_pred = pd.DataFrame()
X_pred['sog'] = np.linspace(data['sog'].min(), data['sog'].max(), 100)
X_pred = features(X_pred)

prediction = X_pred.copy()
prediction['fuel_rate'] = model.predict(X_pred)
prediction['fuel_per_m'] = prediction['fuel_rate']/X_pred['sog']

In [ ]:
fig,ax=plt.subplots()
data_running['fuel_per_m'] = data_running['fuel_rate']/data_running['sog']
data_running.resample('10S').mean().plot(x='sog', y='fuel_per_m', style='.', alpha=0.05, ax=ax, label='__no_label__')

ax.set_ylim(data_running['fuel_per_m'].quantile(0.01), 
            data_running['fuel_per_m'].quantile(0.90))
ax.plot([],'b.', label='fuel rate per m')

prediction.plot(x='sog', y='fuel_per_m', ax=ax, label='regression')
ax.set_xticks(np.arange(0,int(np.ceil(data['sog'].max())),1));
ax.grid(True)
ax.legend()

In [ ]:

#df_ = data_running.resample('10S').mean().dropna()
df_ = data_running

mask = ((df_['fuel_per_m'] > data_running['fuel_per_m'].quantile(0.01)) & 
        (df_['fuel_per_m'] < data_running['fuel_per_m'].quantile(0.90))
       )
df_ = df_.loc[mask].copy()

x = df_['sog']
y = df_['fuel_per_m']
[H, xedges, yedges] = np.histogram2d(y, x, bins=50, density=True)

In [ ]:
X,Y = np.meshgrid((yedges[1:] + yedges[0:-1])/2, (xedges[1:] + xedges[0:-1])/2)

In [ ]:
fig,axes=plt.subplots(nrows=2)
ax =axes[0]
ax.contourf(X,Y,H, levels=20, cmap='gray_r', antialiased=False, label='data')
ax.plot([],'k.', label='fuel rate per m')

prediction.plot(x='sog', y='fuel_per_m', style='k-', ax=ax, label='regression')
ax.set_xticks(np.arange(0,int(np.ceil(data['sog'].max())),1));
ax.set_xticklabels([]);
ax.set_xlabel('')
ax.grid(True)
ax.legend()

ax = axes[1]
data_running['sog'].hist(bins=50, density=True, ax=ax)
ax.set_xticks(np.arange(0,int(np.ceil(data['sog'].max())),1));
ax.set_xlabel('sog [m/s]');
ax.set_ylabel('Occurance')

In [ ]:
import geopandas
data = geopandas.GeoDataFrame(data_running, 
                 geometry=geopandas.points_from_xy(data_running.longitude, data_running.latitude, crs="EPSG:4326"))
data = data.to_crs(epsg=3006)
data['distance'] = data.distance(data.shift())

In [ ]:
data['distance'].sum()

In [ ]:
data.plot()